In [2]:
import numpy  as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import os


In [3]:
!pip install -U pylibjpeg pylibjpeg-openjpeg pylibjpeg-libjpeg pydicom python-gdcm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.0 MB/s eta 0:00:0000:0100:01


## Import des bases 

In [4]:
### Création d'un path de lecture des données ###
path = '/kaggle/input/rsna-breast-cancer-detection/'

train_path = '/kaggle/input/rsna-breast-cancer-detection/train_images'
test_path = '/kaggle/input/rsna-breast-cancer-detection/test_images'

In [5]:
### Import des données d'entrainements ###
train_set = pd.read_csv(path+"train.csv")
train_set.sort_values(by=['patient_id', 'image_id'], inplace=True)
train_set.reset_index(drop=True, inplace=True)

In [6]:
train_set

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,5,640805896,L,MLO,68.0,0,0,0,0.0,0,NaN,21,True
1,2,5,940388076,R,CC,68.0,0,0,0,NaN,0,NaN,21,False
2,2,5,1351088028,L,CC,68.0,0,0,0,0.0,0,NaN,21,True
3,2,5,1633417959,R,MLO,68.0,0,0,0,NaN,0,NaN,21,False
4,1,25,822390278,R,CC,62.0,0,0,0,1.0,0,B,49,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,65530,1627492472,L,MLO,48.0,0,0,0,NaN,0,C,49,False
54702,1,65534,44480502,R,MLO,46.0,0,0,0,1.0,0,C,49,False
54703,1,65534,1174089649,R,CC,46.0,0,0,0,1.0,0,C,49,False
54704,1,65534,1436834199,L,CC,46.0,0,0,0,1.0,0,C,49,False


In [7]:
import pydicom
from os import listdir

def load_patient_scans(path, patient_id):
    patient_path = path + '/' + str(patient_id)
    return [pydicom.dcmread(patient_path + '/' + file) for file in listdir(patient_path)]

In [8]:
# # scans_full_info = []
# scans_matrix = []
# compteur = 0

# ids = train_set['patient_id'].unique()

# for patient_ids in ids[0:10]:
#     # Load all scans of the twelfth patient
#     scans = load_patient_scans(train_path, patient_ids)
#     if compteur % 20 == 0:
#         print('Patient {},  id : {}'.format(compteur, patient_ids))
#     scans_matrix += [scan.pixel_array for scan in scans]
#     del scans
#     compteur += 1

In [9]:
# # Look at raw pixelarrays
# fig, ax = plt.subplots(1, 2, figsize=(20, 5))
# im = ax[0].imshow(scans[0].pixel_array, cmap='bone')
# ax[0].grid(False)
# fig.colorbar(im, ax=ax[0])
# sns.histplot(scans[0].pixel_array.flatten(), ax=ax[1], bins=50)
# plt.show()

### Pre-Processing des images 

Une des premières étapes consistent à pre-processé les données, afin de pouvoir permettre : :
1. Gérer la taille importante des données ;
2. Avoir une meilleure qualité de prédiction sur le modèle.

Pour faire le pré-processing, un certain nombre d'opération vont devoir être réalisées : 
1. Normalisation des couleurs ; 
2. Mise du même coté de toutes les images ; 
3. Suppression de l'arrière plan sur les images ; 
4. *Windowing* : amélioration du contraste pour une meilleure visualisation ;
5. Crop : Delete random part of this image.


Ces idées sont inspirées du NoteBook Kaggle suivant : https://www.kaggle.com/code/paulbacher/custom-preprocessor-rsna-breast-cancer


In [10]:
import cv2
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers import apply_windowing

class MammographyPreProcessing():
    """
    Class whose purpose is to pre-process the data in order to facilitate the analysis.
    """
    
    def __init__(self, size, side):
        """
        Initialization of the class.
        
        Inputs :
            size : Tuple of int.
                Final size of the image to process. If none, can't resize the image.
            side : String.
                Says the size requiered.
        """
        self.size = size
        assert type(size) == tuple, "Size should be a tuple"
        assert len(size) == 2, "Size should be a tuple with a length of two."
        self.side = side
        assert side in ['L', 'R'], "Side should be 'L' or 'R'"
    
    def multiple_preprocessing(self, paths, final_size= 255, save=False, n_jobs=-1):
        """
        Given a list of paths to find images, realize every single preprocessing.
        
        Inputs :
            paths : List of str.
                List of paths to preprocess.
            final_size : int.
                Max of the number when reducing dimension of the image.
            save : bool.
                If save is needed.
            n_jobs : int.
                Number of heart used at the same time for parralelization. If -1, use all.
        """
        
        Parallel(n_jobs=n_jobs)(delayed(self.preprocess_one_image)(path, final_size, save) for path in tqdm(paths, total=len(paths)))
        
    
    def preprocess_one_image(self, path : str, final_size=255, save=False):
        """
        Pre-processing of a single image.
        
        Inputs: 
            path : String.
                Give a path of a DICOM image, which may be read by the following code.
        """
        assert path.endswith('.dcm'), 'Path should provide a DCM image, either it wont work.'
        scan = pydicom.dcmread(path)
        img_matrix = scan.pixel_array
        img_matrix = self._windowing(img_matrix, scan)
        img_matrix = self._fix_photometric_interpretation(img_matrix, scan)
        img_matrix = self._normalize(img_matrix, final_size)
        img_matrix = self._change_laterality(img_matrix)
        img_matrix = self._crop(img_matrix)

        
        img_matrix = cv2.resize(img_matrix, self.size) # Resizing 
        
        if save:
            self.save_image(path, img_matrix)
        
        return img_matrix
    
    def save_image(self, path, img):
        """
        Function to save data after processing it.
        
        Inputs : 
            path : String.
                Give a path of a DICOM image, which may be read by the following code.                
            img : Numpy Array.
                Matrix of scan related values.
        """
        splitted_path = path.split('/')
        path_ = "/kaggle/working/{}".format(splitted_path[-2])

        if not os.path.exists(path_):
            os.makedirs(path_)
        
        with open(path_+'/'+splitted_path[-1].split(".dcm")[0]+'.pkl', 'wb') as f:
            pickle.dump(img, f)
    
    def display_image(self, path, preprocess=False):
        """
        Display the image with a colorful representation.
        
        Inputs:
            img : Numpy Array.
                Matrix of scan related values.
        """
        
        plt.figure(figsize=(8,8))
        if preprocess:
            img = self.preprocess_one_image(path)
        else:
            img = pydicom.dcmread(path).pixel_array
        
        plt.imshow(img, cmap='jet')
        plt.show()
        

    def _windowing(self, img, scan):
        """
        Improve contrast on the picture. This is one of the most relevant part.
        
        Inputs : 
            img : Numpy Array.
                Matrix of scan related values.
            scan : pydicom.dataset.FileDataset.
                Scan related information, with image and several other info.
        """
        return apply_windowing(img, scan)
    
    def _normalize(self, img, final_size=255):
        """
        Normalize the data to put them in grey-scale.
        
        Inputs:
            img : Numpy Array.
                Matrix of scan related values.
            final_size : int.
                Maximum of the pixel matrix value after normalization.
        """
        if img.max()!= 0:
            img /= img.max()
        img *= final_size
        return img
    
    def _change_laterality(self, img):
        """
        Determine laterality of the scan, then apply a changement if the requiere laterality is not the good one.
        
        Inputs : 
            img : Numpy Array.
                Matrix of scan related values.
        """
        middle = len(img[0])//2
        lat = ''
        if sum(sum(img[:, :middle]))>sum(sum(img[:, middle:])):
            lat='L'
        else:
            lat='R'

        if self.side != lat:
            img = np.array(list(map(lambda u: u[::-1], img)))
        
        return img
    
    def _fix_photometric_interpretation(self, img, scan):
        """
        AUTHOR : Paul BACHER.
        Fixing one only colour for scans. "Interpret pixels in a consistant way."
        
        Inputs : 
            img : Numpy Array.
                Matrix of scan related values.
            scan : pydicom.dataset.FileDataset.
                Scan related information, with image and several other info.
        """
        if scan.PhotometricInterpretation == 'MONOCHROME1':
            return img.max() - img
        elif scan.PhotometricInterpretation == 'MONOCHROME2':
            return img - img.min()
        else:
            raise ValueError("Invalid Photometric Interpretation: {}"
                               .format(scan.PhotometricInterpretation))
        
    def _crop(self, img, threshold=5):
        """
        Crop the useless part of the picture.

        Inputs : 
            img : Numpy Array.
                Matrix of scan related values.
            threshold : int.
                Min value that you can decide to delete, when deleting background.
        """
        # Binarization
#         plt.imshow(img)
        bin_img =  (img > threshold).astype(np.uint8)

        # Contouring
        contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contour = max(contours, key=cv2.contourArea)

        # Separation 
        mask = np.zeros(img.shape, np.uint8)
        img_temp = cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED) # Draw contours
        output = cv2.bitwise_and(img_temp, mask) # Fusion of the two previous images 

        x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
        y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
        x1, x2 = int(0.99 * x1), int(1.01 * x2)
        y1, y2 = int(0.99 * y1), int(1.01 * y2)
        return img[y1:y2, x1:x2]        


In [14]:
ids = train_set['patient_id'].unique()

path_test = [train_path + '/' + str(ids[i]) for i in range(len(ids))]
full_files = []
for j in range(len(path_test)):
    full_files += [path_test[j] + '/' + file for file in listdir(path_test[j])]

In [16]:
MammographyPreProcessing((255, 255), 'L').multiple_preprocessing(full_files, save=True)

  0%|          | 0/54706 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
path_test = train_path + '/' + str(ids[0])
files = [file for file in listdir(path_test)]
files
plt.imshow(MammographyPreProcessing((255, 255), 'L').preprocess_one_image(path_test+'/'+files[2], save=True))

In [ ]:
path_test+'/'+files[0]

In [ ]:
with open('/kaggle/working/test.pkl', 'wb') as f:
    pickle.dump(MammographyPreProcessing((255, 255), 'L').display_image(path_test+'/'+files[0], preprocess=True), f)


In [ ]:
MammographyPreProcessing((255, 255), 'L').display_image(path_test+'/'+files[0])

In [ ]:
type(contour), type(mask)

In [ ]:
type(pydicom.dcmread(path_test+'/'+files[0]))

In [ ]:
a = pydicom.dcmread(path_test+'/'+files[0]).pixel_array
middle = len(a[0])//2
print(a[:, :middle].shape, a.shape)

sum(sum(a[:, :middle]))

In [ ]:
import pydicom
from os import listdir

def load_patient_scans(path, patient_id):
    patient_path = path + '/' + str(patient_id)
    return [pydicom.dcmread(patient_path + '/' + file) for file in listdir(patient_path)]

Une visualisation sur le sujet.

In [ ]:
### Sauvegarde de tous les paths ###
all_path = []

### Sauvegarde de tous les paths utiles ###
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        all_path += [os.path.join(dirname, filename)]

### Filtre sur les données inutiles ###
print('Ok first extract')
all_images = [image for image in all_path if (image.endswith(".dcm") and ("train_images" in image))]

In [ ]:
list_cancer = train_set[train_set['cancer']==1].reset_index(drop=True).image_id.astype(str)
list_cancer
# cancer_image = [u for u in list_cancer 

In [ ]:
from gdcm import ImageToPixmapFilter

# specify your image path
image_path = path + 'train_images/{0}/{1}.dcm'.format(train_set[train_set['cancer']==1].reset_index(drop=True).patient_id[0], 
                                                      1360338805)
reader = gdcm.ImageReader()
reader.SetFileName(image_path)
image = reader.Read()

# Convert the image to a pixmap
image_to_pixmap = gdcm.ImageToPixmapFilter()
image_to_pixmap.SetInput(image)
pixmap = image_to_pixmap.GetOutput()

# Get the pixel matrix
pixel_matrix = pixmap.GetBuffer()

print(pixel_matrix)

# print(image)

In [ ]:
import gdcm

image_path = path + 'train_images/{0}/{1}.dcm'.format(train_set[train_set['cancer']==1].reset_index(drop=True).patient_id[0], 
                                                      1360338805)

# Read the DICOM image using the ImageReader
reader = gdcm.ImageReader()
reader.SetFileName(image_path)
image = reader.Read()

# Write the image to a file using the ImageWriter
writer = gdcm.ImageWriter()
writer.SetFileName("output.png")
writer.SetFile(reader.GetFile())
writer.SetImage(image)
writer.Write()

# Print the image
print(image)


In [ ]:
image

In [ ]:
import pylibjpeg
from pydicom import dcmread
from pydicom.pixel_data_handlers import pillow_handler
pillow_handler.PillowJPEGTransferSyntaxes.append('1.2.840.10008.1.2.4.70')
import matplotlib.pylab as plt


print(image_path)
ds = dicom.dcmread(image_path)
# print(ds)
ds.pixel_array
print("La taille de l'image est de {} x {}".format(ds.pixel_array.shape[0], ds.pixel_array.shape[1]))

In [ ]:
ds.pixel_array

In [ ]:
import numpy as np 
np.unique(ds.pixel_array)

#### Entrainement d'un premier modèle très basique 

In [ ]:
import tensorflow as tf

In [ ]:
# Define the model
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, kernel_size=3, stride=1, padding=1)
        self.conv2 = tf.keras.layers.Conv2D(32, kernel_size=3, stride=1, padding=1)
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128)
        self.fc2 = tf.keras.layers.Dense(10)

    def call(self, x):
        x = tf.nn.relu(self.conv1(x))
        x = tf.nn.max_pool2d(x, kernel_size=2, strides=2)
        x = tf.nn.relu(self.conv2(x))
        x = tf.nn.max_pool2d(x, kernel_size=2, strides=2)
        x = self.flatten(x)
        x = tf.nn.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Create the model
model = CNN()

# Define the loss function and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the metrics to track
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Training loop
for epoch in range(5):
    # Training
    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            logits = model(images, training=True)
            loss_value = loss_fn(labels, logits)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric(labels, logits)
    train_acc = train_acc_metric.result()
    train_acc_metric.reset_states()

    # Validation
    for val_images, val_labels in val_dataset:
        val_logits = model(val_images, training=False)
        val_acc_metric(val_labels, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()

    template = "Epoch {}, Train accuracy: {}, Validation accuracy: {}"
    print(template.format(epoch+1, train_acc, val_acc))

# Testing
for test_images, test_labels in test_dataset:
    logits = model(test_images, training=False)
    test_acc_metric(test_labels, logits)
print("Test accuracy: {}".format(test_acc_metric.result()))